In [1]:
from fyers_apiv3 import fyersModel

In [2]:
# Replace these values with your actual API credentials
redirect_uri = "https://9cc0-106-213-83-154.ngrok-free.app"
client_id = "17NRCJWB0H-100"
secret_key = "E28DAAIIND"
grant_type = "authorization_code"
response_type = "code"  
state = "sample_state"

# Create a session model with the provided credentials
session = fyersModel.SessionModel(
    client_id=client_id,
    redirect_uri=redirect_uri,
    response_type=response_type,
    state=state,    
    secret_key=secret_key,
    grant_type=grant_type
)

# Generate the auth code using the session model
response = session.generate_authcode()

# Print the auth code received in the response
print(response)

https://api-t1.fyers.in/api/v3/generate-authcode?client_id=17NRCJWB0H-100&redirect_uri=https%3A%2F%2F9cc0-106-213-83-154.ngrok-free.app&response_type=code&state=sample_state


In [5]:
auth_code = "eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJhcHBfaWQiOiIxN05SQ0pXQjBIIiwidXVpZCI6ImZkZmFkMTc0OGE4YzQ0YTdhMDdhZjA3NTNlOTM2YTg2IiwiaXBBZGRyIjoiIiwibm9uY2UiOiIiLCJzY29wZSI6IiIsImRpc3BsYXlfbmFtZSI6IkZBQTA1ODIwIiwib21zIjoiSzEiLCJoc21fa2V5IjoiMTY3YzkwNDFlOTU2NTg3NWYxMjFmMWJmYzBiNmI0MTk2MDdkZjYyNjg3NGZkYTM2YjY1MTA1M2IiLCJpc0RkcGlFbmFibGVkIjoiTiIsImlzTXRmRW5hYmxlZCI6Ik4iLCJhdWQiOiJbXCJkOjFcIixcImQ6MlwiLFwieDowXCIsXCJ4OjFcIixcIng6MlwiXSIsImV4cCI6MTc1NjU3OTYxOSwiaWF0IjoxNzU2NTQ5NjE5LCJpc3MiOiJhcGkubG9naW4uZnllcnMuaW4iLCJuYmYiOjE3NTY1NDk2MTksInN1YiI6ImF1dGhfY29kZSJ9.1clM0BiS63A2LOqQfnsybBMNvU_8JcNygGnpeSwoUGk"
session.set_token(auth_code)
response = session.generate_token()

try: 
    access_token = response["access_token"]
except Exception as e:
    print(e,response)

In [6]:
print(access_token)

eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJhdWQiOlsiZDoxIiwiZDoyIiwieDowIiwieDoxIiwieDoyIl0sImF0X2hhc2giOiJnQUFBQUFCb3N0SlpRczZGOWFkTEFWaFdGQzJ2cFRPYW5xY2JWLURTTDkweVhWODA3UVVGRWpCZGc2X2o1QnZlX2xaa0pObzZfb3hqNFpTUTZ3MTkxRzZoTG5vTXFPY08weW9RUjVfc0dETm1aT3BMaUlHV3lNND0iLCJkaXNwbGF5X25hbWUiOiIiLCJvbXMiOiJLMSIsImhzbV9rZXkiOiIxNjdjOTA0MWU5NTY1ODc1ZjEyMWYxYmZjMGI2YjQxOTYwN2RmNjI2ODc0ZmRhMzZiNjUxMDUzYiIsImlzRGRwaUVuYWJsZWQiOiJOIiwiaXNNdGZFbmFibGVkIjoiTiIsImZ5X2lkIjoiRkFBMDU4MjAiLCJhcHBUeXBlIjoxMDAsImV4cCI6MTc1NjYwMDIwMCwiaWF0IjoxNzU2NTQ5NzIxLCJpc3MiOiJhcGkuZnllcnMuaW4iLCJuYmYiOjE3NTY1NDk3MjEsInN1YiI6ImFjY2Vzc190b2tlbiJ9.h2RbzAm9BDXRQHowKNYa-LgkJ79oEmpQMF0B1g6CA5I


In [7]:
fyers = fyersModel.FyersModel(token=access_token,is_async=False,client_id=client_id,log_path="")

In [12]:
import datetime
referenceTime = datetime.datetime(2025, 8, 18, 9, 30, 0, 0)
from fyers_apiv3.FyersWebsocket import data_ws

In [13]:
def onmessage(message):
    if 'ltp' not in message.keys():
        return
    
    #time = datetime.datetime.fromtimestamp(message['last_traded_time']/1000.0)

    # print(time)

    if datetime.datetime.now().time() > referenceTime.time():
        if message['ltp'] > message['open_price']+(message['open_price']*0.01):
            print(message['symbol'],"Put Buy Trade")
        elif message['ltp'] < message['open_price']-(message['open_price']*0.01):
            print(message['symbol'],"Put Sell Trade")
        else:
            print(message['symbol'],"Not Tradable")

    
    

def onerror(message):
    """
    Callback function to handle WebSocket errors.

    Parameters:
        message (dict): The error message received from the WebSocket.


    """
    print("Error:", message)


def onclose(message):
    """
    Callback function to handle WebSocket connection close events.
    """
    print("Connection closed:", message)


def onopen():
    """
    Callback function to subscribe to data type and symbols upon WebSocket connection.

    """
    # Specify the data type and symbols you want to subscribe to
    data_type = "SymbolUpdate"
    # data_type = "DepthUpdate"


    # Subscribe to the specified symbols and data type
    symbols = ['NSE:ADANIENT-EQ','NSE:ADANIPORTS-EQ','NSE:APOLLOHOSP-EQ','NSE:ASIANPAINT-EQ','NSE:AXISBANK-EQ','NSE:MARUTI-EQ']
    fyers.subscribe(symbols=symbols, data_type=data_type)

    # Keep the socket running to receive real-time data
    fyers.keep_running()

In [14]:
ws_access_token = "17NRCJWB0H-100:"+access_token

fyers = data_ws.FyersDataSocket(
    access_token=ws_access_token,       # Access token in the format "appid:accesstoken"
    log_path="",                     # Path to save logs. Leave empty to auto-create logs in the current directory.
    litemode=False,                  # Lite mode disabled. Set to True if you want a lite response.
    write_to_file=False,              # Save response in a log file instead of printing it.
    reconnect=True,                  # Enable auto-reconnection to WebSocket on disconnection.
    on_connect=onopen,               # Callback function to subscribe to data upon connection.
    on_close=onclose,                # Callback function to handle WebSocket connection close events.
    on_error=onerror,                # Callback function to handle WebSocket errors.
    on_message=onmessage             # Callback function to handle incoming messages from the WebSocket.
)

# Establish a connection to the Fyers WebSocket
fyers.connect()

NSE:MARUTI-EQ Not Tradable
NSE:AXISBANK-EQ Not Tradable
NSE:ADANIENT-EQ Put Sell Trade
NSE:ADANIPORTS-EQ Not Tradable
NSE:ASIANPAINT-EQ Put Buy Trade
NSE:APOLLOHOSP-EQ Not Tradable
Connection closed: {'code': 200, 'message': 'Connection Closed', 's': 'ok'}


In [15]:
fyers.close_connection()